In [240]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import data_utils
import train_utils
import models

%reload_ext autoreload
%autoreload 2

In [22]:
dev = data_utils.COAD_dataset(data_utils.COAD_DEV)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=1, shuffle=True , pin_memory=True)

In [253]:
n_conv_layers = 2
n_fc_layers = 2
kernel_size = [4,3]
n_conv_filters = [36,48]
hidden_size = [512,512]
dropout=0.5
net = models.ConvNet(n_conv_layers, n_fc_layers, kernel_size, n_conv_filters, hidden_size, dropout=dropout)
net.cuda()

ConvNet(
  (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (n): Dropout(p=0.5)
  (relu): ReLU()
  (conv): Sequential(
    (0): Conv2d(3, 36, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(36, 48, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1200, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5)
  )
  (classification_layer): Linear(in_features=512, out_features=2, bias=True)
)

In [254]:
epochs = 100 
criterion = nn.CrossEntropyLoss()

In [255]:
lr = 0.0001
weight_decay = 0.0005

In [250]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

In [243]:
train = data_utils.COAD_dataset(data_utils.COAD_TRAIN)
train_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True , pin_memory=True)

valid = data_utils.COAD_dataset(data_utils.COAD_VALID)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=1, shuffle=True , pin_memory=True)

In [182]:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=4)

In [185]:
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v

best_loss = 1e8
for e in range(epochs):
    train_utils.embedding_training_loop(e, train_loader, net, criterion, optimizer,pool_fn)
    loss = train_utils.embedding_validation_loop(e, valid_loader, net, criterion,pool_fn)
    #scheduler.step(loss)
    print('LR = {}'.format(optimizer.state_dict()['param_groups'][0]['lr']))
    if loss < best_loss:
        torch.save(net.state_dict(),'best_convnet.pt')
        best_loss = loss
        print('WROTE MODEL')

Epoch: 0, Train NLL: 34.7338
Epoch: 0, Val NLL: 34.2249, Val Acc: 0.4082
LR = 0.0001
WROTE MODEL
Epoch: 1, Train NLL: 34.9582
Epoch: 1, Val NLL: 34.6048, Val Acc: 0.4082
LR = 0.0001
Epoch: 2, Train NLL: 34.5777
Epoch: 2, Val NLL: 34.9021, Val Acc: 0.4082
LR = 0.0001
Epoch: 3, Train NLL: 34.5222
Epoch: 3, Val NLL: 34.5258, Val Acc: 0.4082
LR = 0.0001
Epoch: 4, Train NLL: 34.3403
Epoch: 4, Val NLL: 34.9956, Val Acc: 0.4082
LR = 0.0001
Epoch: 5, Train NLL: 34.3574
Epoch: 5, Val NLL: 35.1883, Val Acc: 0.4082
LR = 0.0001
Epoch: 6, Train NLL: 34.3542
Epoch: 6, Val NLL: 34.9374, Val Acc: 0.4082
LR = 0.0001
Epoch: 7, Train NLL: 34.1592
Epoch: 7, Val NLL: 35.0457, Val Acc: 0.4082
LR = 0.0001
Epoch: 8, Train NLL: 34.6030
Epoch: 8, Val NLL: 34.4113, Val Acc: 0.4082
LR = 0.0001
Epoch: 9, Train NLL: 34.4462
Epoch: 9, Val NLL: 34.1298, Val Acc: 0.4082
LR = 0.0001
WROTE MODEL
Epoch: 10, Train NLL: 34.1670
Epoch: 10, Val NLL: 34.7604, Val Acc: 0.4082
LR = 0.0001
Epoch: 11, Train NLL: 33.8684
Epoch: 11

Epoch: 93, Train NLL: 0.0510
Epoch: 93, Val NLL: 15.1567, Val Acc: 0.8571
LR = 0.0001
Epoch: 94, Train NLL: 0.0536
Epoch: 94, Val NLL: 15.3927, Val Acc: 0.8571
LR = 0.0001
Epoch: 95, Train NLL: 0.0487
Epoch: 95, Val NLL: 15.2665, Val Acc: 0.8571
LR = 0.0001
Epoch: 96, Train NLL: 0.0415
Epoch: 96, Val NLL: 15.4121, Val Acc: 0.8571
LR = 0.0001
Epoch: 97, Train NLL: 0.0411
Epoch: 97, Val NLL: 15.6417, Val Acc: 0.8571
LR = 0.0001
Epoch: 98, Train NLL: 0.0410
Epoch: 98, Val NLL: 15.5140, Val Acc: 0.8571
LR = 0.0001
Epoch: 99, Train NLL: 0.0371
Epoch: 99, Val NLL: 15.9755, Val Acc: 0.8776
LR = 0.0001


In [191]:
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v

best_loss = 1e8
for e in range(epochs):
    train_utils.instance_training_loop(e, train_loader, net, criterion, optimizer,pool_fn)
    loss = train_utils.instance_validation_loop(e, valid_loader, net, criterion,pool_fn)
    #scheduler.step(loss)
    print('LR = {}'.format(optimizer.state_dict()['param_groups'][0]['lr']))
    if loss < best_loss:
        torch.save(net.state_dict(),'best_convnet.pt')
        best_loss = loss
        print('WROTE MODEL')

Epoch: 0, Train NLL: 34.7253
Epoch: 0, Val NLL: 34.5098, Val Acc: 0.4082
LR = 0.0001
WROTE MODEL
Epoch: 1, Train NLL: 34.4373
Epoch: 1, Val NLL: 34.9781, Val Acc: 0.4082
LR = 0.0001
Epoch: 2, Train NLL: 34.5094
Epoch: 2, Val NLL: 34.7606, Val Acc: 0.4082
LR = 0.0001
Epoch: 3, Train NLL: 34.5293
Epoch: 3, Val NLL: 34.8371, Val Acc: 0.4082
LR = 0.0001
Epoch: 4, Train NLL: 34.3855
Epoch: 4, Val NLL: 34.7604, Val Acc: 0.4082
LR = 0.0001
Epoch: 5, Train NLL: 34.3665
Epoch: 5, Val NLL: 34.9662, Val Acc: 0.4082
LR = 0.0001
Epoch: 6, Train NLL: 34.3384
Epoch: 6, Val NLL: 34.5777, Val Acc: 0.4082
LR = 0.0001
Epoch: 7, Train NLL: 34.4144
Epoch: 7, Val NLL: 34.7917, Val Acc: 0.4082
LR = 0.0001
Epoch: 8, Train NLL: 34.3746
Epoch: 8, Val NLL: 34.8322, Val Acc: 0.4082
LR = 0.0001
Epoch: 9, Train NLL: 34.3017
Epoch: 9, Val NLL: 34.5868, Val Acc: 0.4082
LR = 0.0001
Epoch: 10, Train NLL: 34.4337
Epoch: 10, Val NLL: 34.3766, Val Acc: 0.4082
LR = 0.0001
WROTE MODEL
Epoch: 11, Train NLL: 34.4705
Epoch: 11

Epoch: 90, Train NLL: 9.8912
Epoch: 90, Val NLL: 5.6295, Val Acc: 0.9592
LR = 0.0001
Epoch: 91, Train NLL: 10.3956
Epoch: 91, Val NLL: 5.7194, Val Acc: 0.9592
LR = 0.0001
Epoch: 92, Train NLL: 10.2883
Epoch: 92, Val NLL: 4.0148, Val Acc: 1.0000
LR = 0.0001
WROTE MODEL
Epoch: 93, Train NLL: 11.6157
Epoch: 93, Val NLL: 4.6198, Val Acc: 1.0000
LR = 0.0001
Epoch: 94, Train NLL: 9.5605
Epoch: 94, Val NLL: 5.2011, Val Acc: 0.9592
LR = 0.0001
Epoch: 95, Train NLL: 10.8057
Epoch: 95, Val NLL: 3.4065, Val Acc: 1.0000
LR = 0.0001
WROTE MODEL
Epoch: 96, Train NLL: 10.0294
Epoch: 96, Val NLL: 6.4258, Val Acc: 0.9388
LR = 0.0001
Epoch: 97, Train NLL: 10.6172
Epoch: 97, Val NLL: 4.5113, Val Acc: 0.9592
LR = 0.0001
Epoch: 98, Train NLL: 10.2490
Epoch: 98, Val NLL: 3.9005, Val Acc: 1.0000
LR = 0.0001
Epoch: 99, Train NLL: 9.6537
Epoch: 99, Val NLL: 3.8658, Val Acc: 1.0000
LR = 0.0001


In [262]:
input_size = 512
hidden_size = 512
output_size = 1
attn = models.Attention(input_size, hidden_size, output_size)
attn.cuda()
#for slide,label in dev_loader:
#    break

#h = net(slide.squeeze_().cuda())
#a = attn(h)

Attention(
  (V): Linear(in_features=512, out_features=512, bias=True)
  (U): Linear(in_features=512, out_features=512, bias=True)
  (w): Linear(in_features=512, out_features=1, bias=True)
  (sigm): Sigmoid()
  (tanh): Tanh()
  (sm): Softmax()
)

In [210]:
h.shape

torch.Size([196, 512])

In [211]:
a.shape

torch.Size([196, 1])

In [216]:
z = torch.transpose(a, dim0=0, dim1=1).matmul(h)

In [217]:
output = net.classification_layer(z)
output

tensor([[-4.4068,  4.2534]], device='cuda:0', grad_fn=<AddmmBackward>)

In [258]:
parameters = [p for p in net.parameters()]
print(len(parameters))
parameters.extend([p for p in attn.parameters()])
print(len(parameters))

10
16


In [259]:
optimizer = torch.optim.Adam(parameters, lr=lr, weight_decay=weight_decay)

In [263]:
pool_fn = models.pool(attn)

In [261]:
best_loss = 1e8
for e in range(epochs):
    train_utils.embedding_training_loop(e, train_loader, net, criterion, optimizer,pool_fn)
    loss = train_utils.embedding_validation_loop(e, valid_loader, net, criterion,pool_fn)
    #scheduler.step(loss)
    print('LR = {}'.format(optimizer.state_dict()['param_groups'][0]['lr']))
    if loss < best_loss:
        torch.save(net.state_dict(),'best_convnet.pt')
        best_loss = loss
        print('WROTE MODEL')

Epoch: 0, Train NLL: 34.5292
Epoch: 0, Val NLL: 34.7943, Val Acc: 0.4082
LR = 0.0001
WROTE MODEL
Epoch: 1, Train NLL: 34.4987
Epoch: 1, Val NLL: 35.6473, Val Acc: 0.4082
LR = 0.0001
Epoch: 2, Train NLL: 34.5297
Epoch: 2, Val NLL: 35.1216, Val Acc: 0.4082
LR = 0.0001
Epoch: 3, Train NLL: 34.4356
Epoch: 3, Val NLL: 35.0022, Val Acc: 0.4082
LR = 0.0001
Epoch: 4, Train NLL: 34.3379
Epoch: 4, Val NLL: 35.0881, Val Acc: 0.4082
LR = 0.0001
Epoch: 5, Train NLL: 34.5551
Epoch: 5, Val NLL: 35.4290, Val Acc: 0.4082
LR = 0.0001
Epoch: 6, Train NLL: 34.4301
Epoch: 6, Val NLL: 34.9031, Val Acc: 0.4082
LR = 0.0001
Epoch: 7, Train NLL: 34.3130
Epoch: 7, Val NLL: 35.0679, Val Acc: 0.4082
LR = 0.0001
Epoch: 8, Train NLL: 34.4213
Epoch: 8, Val NLL: 35.0104, Val Acc: 0.4082
LR = 0.0001
Epoch: 9, Train NLL: 34.2849
Epoch: 9, Val NLL: 35.0321, Val Acc: 0.4082
LR = 0.0001
Epoch: 10, Train NLL: 34.4514
Epoch: 10, Val NLL: 34.9220, Val Acc: 0.4082
LR = 0.0001
Epoch: 11, Train NLL: 34.3581
Epoch: 11, Val NLL: 3

Epoch: 90, Train NLL: 14.8006
Epoch: 90, Val NLL: 11.5867, Val Acc: 0.8980
LR = 0.0001
Epoch: 91, Train NLL: 15.4109
Epoch: 91, Val NLL: 10.3784, Val Acc: 0.8980
LR = 0.0001
WROTE MODEL
Epoch: 92, Train NLL: 14.1421
Epoch: 92, Val NLL: 27.4158, Val Acc: 0.7143
LR = 0.0001
Epoch: 93, Train NLL: 18.0540
Epoch: 93, Val NLL: 19.4861, Val Acc: 0.7959
LR = 0.0001
Epoch: 94, Train NLL: 14.2041
Epoch: 94, Val NLL: 11.1944, Val Acc: 0.9592
LR = 0.0001
Epoch: 95, Train NLL: 12.2540
Epoch: 95, Val NLL: 10.4253, Val Acc: 0.9184
LR = 0.0001
Epoch: 96, Train NLL: 13.3577
Epoch: 96, Val NLL: 12.2173, Val Acc: 0.9184
LR = 0.0001
Epoch: 97, Train NLL: 11.7640
Epoch: 97, Val NLL: 10.9356, Val Acc: 0.9388
LR = 0.0001
Epoch: 98, Train NLL: 10.4431
Epoch: 98, Val NLL: 10.3714, Val Acc: 0.8776
LR = 0.0001
WROTE MODEL
Epoch: 99, Train NLL: 14.6352
Epoch: 99, Val NLL: 9.7411, Val Acc: 0.9184
LR = 0.0001
WROTE MODEL


In [233]:
net

ConvNet(
  (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (n): Dropout(p=0)
  (relu): ReLU()
  (conv): Sequential(
    (0): Conv2d(3, 36, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(36, 48, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1200, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0)
  )
  (classification_layer): Linear(in_features=512, out_features=2, bias=True)
)